## Test flatten()

In [2]:
for i in range(2):
    print(i)

0
1


In [21]:
import torch

test = torch.randn(10, 3)
flattened_test = test.flatten()
global_mean_pooled = torch.mean(test, dim = 0)
print(f"test (shape: {test.shape}): \n{test}\n\nglobal_mean_pooled (shape: {global_mean_pooled.shape}): \n{global_mean_pooled}")

test (shape: torch.Size([10, 3])): 
tensor([[ 0.1535,  1.7894, -1.6731],
        [-0.2661,  0.0507,  0.0122],
        [-0.8373, -1.2231, -0.2125],
        [ 0.1431, -0.3771,  0.2808],
        [-0.6767, -1.2800,  1.6357],
        [-1.2809,  0.4099,  1.0182],
        [ 0.1278,  0.2539, -2.3672],
        [-0.9886, -0.2926,  0.6868],
        [ 1.4347,  0.8221,  0.2358],
        [-0.4437, -0.8436,  0.7986]])

global_mean_pooled (shape: torch.Size([3])): 
tensor([-0.2634, -0.0690,  0.0415])


## Check F.MSELoss vs torch.nn.MSELoss
Turns out F.MSELoss is averaged, just like torch.nn.MSELOss().mean(dim = 0).mean() 

In [13]:
import torch
import torch.nn.functional as F

z_true = torch.randn(10, 3)
z_pred = torch.randn(10, 3)

map_loss = F.mse_loss(z_pred, z_true)

In [14]:
loss = torch.nn.MSELoss(reduction='mean')

In [15]:
print(f"z_true: \n{z_true}")
print(f"z_pred: \n{z_pred}")
print(f"z_true - z_pred: \n{(z_true - z_pred)}")
print(f"z_true - z_pred: \n{(z_true - z_pred)**2}")

z_true: 
tensor([[ 0.0632,  1.3691,  0.3653],
        [-1.1828,  0.9950, -1.8710],
        [-1.7107, -2.0067, -1.7619],
        [-0.3057,  0.3221, -0.6640],
        [ 0.3010,  1.3926, -0.1006],
        [ 0.3183, -2.0068,  1.7918],
        [ 1.1903,  2.5210, -0.7391],
        [-0.8901,  0.6207, -0.3034],
        [ 1.2093,  1.8479, -0.0586],
        [ 1.9255, -0.3426, -1.5023]])
z_pred: 
tensor([[ 0.5558, -1.0083, -0.2440],
        [-0.7069,  0.1746,  0.8102],
        [ 0.9642, -0.5036, -1.0670],
        [-1.6713,  0.0270, -1.2796],
        [ 1.1588, -0.8908,  1.4583],
        [-0.6305, -0.8114,  1.6064],
        [ 0.3240, -0.1243, -0.9219],
        [-1.2829,  0.1069,  0.3512],
        [ 0.1232, -0.2017,  0.7638],
        [-2.6965,  0.6185,  0.5860]])
z_true - z_pred: 
tensor([[-0.4925,  2.3774,  0.6093],
        [-0.4760,  0.8204, -2.6812],
        [-2.6749, -1.5030, -0.6949],
        [ 1.3656,  0.2952,  0.6156],
        [-0.8577,  2.2833, -1.5589],
        [ 0.9488, -1.1954,  0.1854],


In [16]:
loss(z_pred, z_true).mean(dim = 0).mean(dim= -1)

tensor(2.6220)

In [17]:
map_loss

tensor(2.6220)

## Check multi dimension

In [ ]:
import torch_geometric.nn as gnn
import torch
from src.utils.commons import get_config
from src.data.loader_paper import GraphDatasetPaper

config = get_config('configs/paper.yaml')
model = gnn.GMMConv(in_channels=2, out_channels=2, dim=2, kernel_size=5)
dataset = GraphDatasetPaper(config = config['config'], split = 'train')
model(dataset[0].x, dataset[0].edge_index, dataset[0].edge_attr)
# x = torch.randn(10, 2)
# edge_index = torch.randint(0, 10, (2, 10))

# data = Data(x=x, edge_index=edge_index)

# model()
# model()

tensor([[-0.5627, -0.2221],
        [-0.5627, -0.2221],
        [-0.5627, -0.2221],
        ...,
        [-0.5625, -0.2223],
        [-0.5625, -0.2223],
        [-0.5625, -0.2222]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
dataset[0].x

e:\TA\TA2\GAE-ROM\src\data\loader_paper.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(self.edge_features, dtype=torch.float64),
e:\TA\TA2\GAE-ROM\src\data\loader_paper.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_weight = torch.tensor(self.edge_weights, dtype=torch.float64),


tensor([[ 0.7055, -0.5236],
        [ 0.7089, -0.5241],
        [ 0.7134, -0.5247],
        ...,
        [ 0.7068, -0.5237],
        [ 0.7065, -0.5236],
        [ 0.7063, -0.5233]])

In [ ]:
dataset[0].edge_index.shape

e:\TA\TA2\GAE-ROM\src\data\loader_paper.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(self.edge_features, dtype=torch.float64),
e:\TA\TA2\GAE-ROM\src\data\loader_paper.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_weight = torch.tensor(self.edge_weights, dtype=torch.float64),


torch.Size([16229, 3])

## Others

In [1]:
import gc
import torch
from src.utils.commons import get_config, get_device, set_seed
from src.model.gae import GAE
from src.data.loader_paper import GraphDatasetPaper
from warnings import filterwarnings
from torch_geometric.loader import DataLoader
from src.training.train import train

filterwarnings('ignore')
torch.cuda.empty_cache()
gc.collect()

config = get_config('configs/default.yaml')
# change seed
config['config']['seed'] = 42
set_seed(config['config']['seed'])

device = get_device()
train_dataset = GraphDatasetPaper(split = 'test')
val_dataset = GraphDatasetPaper(split = 'val')

model = GAE(config=config).to(device)

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=config['training']['batch_size'], 
                          shuffle=False,
                          num_workers=config['training']['num_workers'])

val_loader = DataLoader(dataset=val_dataset, 
                        batch_size=config['training']['batch_size'], 
                        shuffle=False,
                        num_workers=config['training']['num_workers'])

In [ ]:
train_history = train(model=model, 
                      device=device,
                      train_loader=train_loader,
                      val_loader=val_loader,
                      is_tqdm=False,
                      is_val=True,
                      single_batch=config['training']['single_batch_run'],
                      save_best_model=config['training']['save_best_model'],
                      config=config['training'])

In [4]:
import torch.nn.functional as F
train_dataset = GraphDataset(split = 'train')
model.eval()
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=config['training']['batch_size'], 
                          shuffle=False,
                          num_workers=config['training']['num_workers'])
data = next(iter(train_loader))
target = data.x
predicted, _ = model(data)  # Your model output

In [5]:
dataset[0].edge_attr.shape

e:\TA\TA2\GAE-ROM\src\data\loader_paper.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(self.edge_features, dtype=torch.float64),
e:\TA\TA2\GAE-ROM\src\data\loader_paper.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_weight = torch.tensor(self.edge_weights, dtype=torch.float64),


torch.Size([16230, 2])

In [5]:
loss = F.mse_loss(predicted, target)
print(f"Re-evaluation loss: {loss.item()}")

Re-evaluation loss: 0.5730345845222473


In [4]:
from src.utils.commons import save_model
save_model(model, f'artifacts/{config["config"]["name"]}.pth')


In [3]:
import matplotlib.pyplot as plt

plt.plot(train_history['train_loss'])
plt.plot(train_history['val_loss'])

NameError: name 'train_history' is not defined

In [ ]:
from src.utils.commons import load_model, set_seed, get_device
from src.data.loader import GraphDataset
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

set_seed(42)
device = get_device()
train_dataset = GraphDataset(split = 'train')
model = load_model('artifacts/GAT_100_pool.pth')
model.to(device)
model.eval()
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=config['training']['batch_size'], 
                          shuffle=False,
                          num_workers=config['training']['num_workers'])

batch = next(iter(train_loader)).to(device)
target = batch.x
out, _ = model(batch)
loss = F.mse_loss(out, target)
print(f"Re-evaluation loss: {loss.item()}")

Re-evaluation loss: 0.5067916512489319


In [1]:
from src.utils.commons import load_model, get_config, set_seed
from src.data.loader import GraphDataset
from torch_geometric.loader import DataLoader

config = get_config('configs/default.yaml')
set_seed(config['config']['seed'])

train_dataset = GraphDataset(split = 'train')
model = load_model(save_path = f'artifacts/GAT_100_pool.pth')
model.eval()
data = train_dataset[0]
target = data.x
predicted, _ = model(data)  # Your model output

In [12]:
import torch
print(f"Mean: {torch.mean(target[:,0])}")
print(f"Std: {torch.std(target[:,0])}")
print(f"Max: {torch.max(target[:,0])}")
print(f"Min: {torch.min(target[:,0])}")


Mean: 4.946764109448054e-10
Std: 1.000016212463379
Max: 1.93668794631958
Min: -5.757261753082275


In [13]:
predicted

tensor([[ 2.3000, -0.6171],
        [ 0.6963, -0.6171],
        [ 1.2671, -0.6171],
        ...,
        [-1.3286, -0.6171],
        [-0.2173, -0.6171],
        [-0.7265, -0.4834]], grad_fn=<NativeBatchNormBackward0>)

In [14]:
target

tensor([[ 1.6893, -0.3061],
        [ 0.6079, -0.5585],
        [ 0.8825, -0.5192],
        ...,
        [-0.7824, -0.8180],
        [-0.0327, -0.1216],
        [-0.0343, -0.0886]])

In [2]:
import torch.nn.functional as F
print(F.mse_loss(predicted[:,0], target[:,0]).detach().cpu().numpy())
# print(F.rmse_loss(predicted[:,0], target[:,0]).detach().cpu().numpy())
print(F.l1_loss(predicted[:,0], target[:,0]).detach().cpu().numpy())
print(F.smooth_l1_loss(predicted[:,0], target[:,0]).detach().cpu().numpy())
print(F.huber_loss(predicted[:,0], target[:,0]).detach().cpu().numpy())
print(F.smooth_l1_loss(predicted[:,0], target[:,0]).detach().cpu().numpy())
print(F.l1_loss(predicted[:,0], target[:,0]).detach().cpu().numpy())

0.62439245
0.50935614
0.235132
0.235132
0.235132
0.50935614


In [ ]:
from src.utils.plot import Plot

# Create plotter
plotter = Plot(save_dir='output/')

# After getting model output
plotter.plot_velocity_field(predicted_batch, title="Model Prediction", save=False)

In [ ]:
import matplotlib.pyplot as plt

train_dataset.mesh.points[:,:2]
plt.scatter(train_dataset.mesh.points[:,0], 
            train_dataset.mesh.points[:,1])
# make the aspect ratio 8:4
plt.gca().set_aspect('equal', 'box')
plt.show()


In [ ]:
batch = train_dataset[3]
plt.scatter(batch.pos[:, 0].detach().cpu().numpy(), 
            batch.pos[:, 1].detach().cpu().numpy(), 
            c=batch.x[:, 0].detach().cpu().numpy(), 
            cmap='viridis',
            s=0.05)
plt.gca().set_aspect('equal', 'box')
plt.show()